In [1]:
import pandas
import dask.dataframe

# See: 
#  https://docs.dask.org/en/latest/dataframe-api.html

df = pandas.DataFrame({
    'group': ['a', 'a', 'a', 'b', 'b', 'b'],
    'order': [  1,    3,  2,   6,   9,   5],
    'other_info': [  1,    2,  3,   4,   5,   6]})
dd_orig = dask.dataframe.from_pandas(df, npartitions=2)


In [2]:
# grouped function such as count or row order

group_col = 'group'
order_col = 'order'
temp_col = '_orig_index_column'
result_col = 'per_group_seq'

dd = dd_orig.copy()

dd = dd.reset_index(drop=True)
dd[temp_col] = dd.index
dd = dd.set_index(dd[temp_col])

dsub = dd.loc[:, [group_col, order_col, temp_col]]
dsub = dsub.set_index(dsub[order_col])
dsub[result_col] = dsub.groupby(group_col).cumcount()
dsub = dsub.set_index(dsub[temp_col])
dd[result_col] = dsub[result_col]
dd = dd.reset_index(drop=True)

dd.compute()


,group,order,other_info,_orig_index_column,per_group_seq
0,a,1,1,0,0
1,b,6,4,0,1
0,a,3,2,1,2
1,b,9,5,1,2
2,a,2,3,2,1
3,b,5,6,2,0


In [3]:
# grouped aggregation, merged back into original frame

group_col = 'group'
value_col = 'order'
temp_col = '_orig_index_column'
result_col = 'per_group_sum'

dd = dd_orig.copy()

dd = dd.reset_index(drop=True)
dd[temp_col] = dd.index
dd = dd.set_index(dd[temp_col])

dsub = dd.loc[:, [group_col, value_col, temp_col]]
dagg = dsub.groupby(dsub[group_col]).sum()
dagg = dagg.reset_index(drop = False)
dagg = dagg.set_index(dagg[group_col])
dsub = dsub.set_index(dsub[group_col])
dsub[result_col] = dagg[value_col]
dsub = dsub.set_index(dsub[temp_col])
dd[result_col] = dsub[result_col]
dd = dd.reset_index(drop=True)

dd.compute()

,group,order,other_info,_orig_index_column,per_group_sum
0,b,6,4,0,6
1,a,1,1,0,20
0,b,9,5,1,6
1,a,3,2,1,20
2,b,5,6,2,6
3,a,2,3,2,20
